<a href="https://colab.research.google.com/github/akshitadixit/Retinopathy/blob/main/loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import os
import cv2
import csv
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive")

import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers,Model,Sequential
from keras.layers import Input,Conv2D,Dropout,Dense,Activation, MaxPooling2D, Flatten

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
data_root = "/content/gdrive/My Drive/Colab Notebooks/dataset"
path = os.path.join(data_root, "symp")

with open('/content/gdrive/My Drive/Colab Notebooks/dataset/dataset.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  for i in os.listdir(path):
    writer.writerow([0, os.path.join(path, i)])

In [ ]:
path = os.path.join(data_root, "nosymp")

with open('/content/gdrive/My Drive/Colab Notebooks/dataset/dataset.csv', 'a', newline='') as file:
  writer = csv.writer(file)
  for i in os.listdir(path):
    writer.writerow([1, os.path.join(path, i)])

In [ ]:
# shuffle the data in csv
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/dataset.csv')
ds = df.sample(frac=1)
ds.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/dataset.csv', index=False)

In [ ]:
##############

In [4]:
!pip install split-folders

In [71]:
# split the data

import numpy as np
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_aux/dataset.csv', header=None)
df.columns = ["class", "path"]
df = df.astype({"class": str})
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.7*len(df)), int(.85*len(df))])

In [68]:
len(train["path"])

978

In [78]:
# Model parameters
BATCH_SIZE = 5
EPOCHS = 20
WARMUP_EPOCHS = 2
LEARNING_RATE = 1e-4
WARMUP_LEARNING_RATE = 1e-3
HEIGHT = 500
WIDTH = 500
CANAL = 3
N_CLASSES = 2
ES_PATIENCE = 5
RLROP_PATIENCE = 3
DECAY_DROP = 0.5

In [72]:
train_idg = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_idg = ImageDataGenerator(rescale=1./255)

train_generator = train_idg.flow_from_dataframe(dataframe=train,
                                                      directory=None,
                                                      x_col="path",
                                                      y_col="class",
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical",
                                                      target_size=(HEIGHT, WIDTH))

valid_generator = train_idg.flow_from_dataframe(dataframe=validate,
                                                      directory=None,
                                                      x_col="path",
                                                      y_col="class",
                                                      batch_size=BATCH_SIZE,
                                                      class_mode="categorical",    
                                                      target_size=(HEIGHT, WIDTH))

test_generator = test_idg.flow_from_dataframe(dataframe=test,
                                                      directory=None,
                                                      x_col="path",
                                                      target_size=(HEIGHT, WIDTH),
                                                      batch_size=1,
                                                      shuffle=False,
                                                      class_mode=None)
    


Found 978 validated image filenames belonging to 2 classes.
Found 210 validated image filenames belonging to 2 classes.
Found 210 validated image filenames.


In [75]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(500,500,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [79]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

history_warmup = model.fit_generator(generator=train_generator,
                                     steps_per_epoch=STEP_SIZE_TRAIN,
                                     validation_data=valid_generator,validation_steps=STEP_SIZE_VALID,
                                     epochs=WARMUP_EPOCHS,
                                     verbose=1).history

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
195/195 [==============================] - 1939s 10s/step - loss: 0.6847 - accuracy: 0.5858 - val_loss: 0.6862 - val_accuracy: 0.5619
Epoch 2/2
195/195 [==============================] - 1952s 10s/step - loss: 0.6808 - accuracy: 0.5879 - val_loss: 0.6855 - val_accuracy: 0.5619


In [85]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

es = EarlyStopping(monitor='val_loss', mode='min', patience=ES_PATIENCE, restore_best_weights=True, verbose=1)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=RLROP_PATIENCE, factor=DECAY_DROP, min_lr=1e-6, verbose=1)

callback_list = [es, rlrop]
optimizer = optimizers.Adam(lr=LEARNING_RATE)

In [ ]:
history_finetunning = model.fit_generator(generator=train_generator,
                                          steps_per_epoch=STEP_SIZE_TRAIN,
                                          validation_data=valid_generator,
                                          validation_steps=STEP_SIZE_VALID,
                                          epochs=EPOCHS,
                                          callbacks=callback_list,
                                          verbose=1).history

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
195/195 [==============================] - 1983s 10s/step - loss: 0.6802 - accuracy: 0.5879 - val_loss: 0.6855 - val_accuracy: 0.5619
Epoch 2/20
123/195 [=================>............] - ETA: 11:30 - loss: 0.6772 - accuracy: 0.5905